In [15]:
import pandas as pd
from bs4 import BeautifulSoup
import os

def read_html_to_dataframe(directory):
    data = []
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".html"):
                file_path = os.path.join(root, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        soup = BeautifulSoup(file, 'html.parser')
                        text = soup.get_text()
                        data.append({'path': file_path, 'text': text})
                except UnicodeDecodeError:
                    # Try a different encoding if UTF-8 fails
                    with open(file_path, 'r', encoding='iso-8859-1') as file:
                        soup = BeautifulSoup(file, 'html.parser')
                        text = soup.get_text()
                        data.append({'path': file_path, 'text': text})
    return pd.DataFrame(data)

# Directory containing your HTML files and subdirectories
directory = 'ee.co.uk'
df = read_html_to_dataframe(directory)
df


path  \
0                                   ee.co.uk/help.html   
1                     ee.co.uk/help/home-security.html   
2                          ee.co.uk/help/tv-sport.html   
3                    ee.co.uk/help/cyber-security.html   
4                     ee.co.uk/help/subscriptions.html   
..                                                 ...   
445  ee.co.uk/help/orders/return-and-recycle-ee-pro...   
446  ee.co.uk/help/orders/how-will-my-order-be-deli...   
447  ee.co.uk/help/orders/how-to-return-a-product-b...   
448       ee.co.uk/help/orders/track-mobile-order.html   
449     ee.co.uk/help/orders/consumer-electronics.html   

                                                  text  
0    ﻿\n\n\n\n\n\n\nHelp & Support | EE\n\n\n\n\n\n...  
1    ﻿\n\n\n\n\n\n\nHome Security Help | EE\n\n\n\n...  
2    ﻿\n\n\n\n\n\n\nTV & Sport Help | EE\n\n\n\n\n\...  
3    ﻿\n\n\n\n\n\n\nCyber Security Help | EE\n\n\n\...  
4    ﻿\n\n\n\n\n\n\nHelp with subscriptions through...  
..                                                 ...  
445  ﻿\n\n\n\n  \n\n\n\nRecycle Your EE Equipment f...  
446  ﻿\n\n\n\n  \n\n\n\nHow Will My Phone Be Delive...  
447  ﻿\n\n\n\n  \n\n\n\nReturning a Phone or Device...  
448  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...  
449  ﻿\n\n\n\n  \n\n\n\nElectronics, accessories an...  

[450 rows x 2 columns]

In [17]:
df.to_feather ('ee.co.uk.feather')

In [11]:
from bs4 import BeautifulSoup
import os

def parse_html_to_text(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".html"):
            # Construct full file path
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                # Parse HTML content
                soup = BeautifulSoup(file, 'html.parser')
                text = soup.get_text()

                # Write to a new .txt file
                output_filename = filename.replace('.html', '.txt')
                with open(os.path.join(directory, output_filename), 'w', encoding='utf-8') as output_file:
                    output_file.write(text)

# Directory containing your HTML files
directory = 'downloaded_pages'
parse_html_to_text(directory)


In [9]:
from selenium import webdriver
import os

def fetch_and_save(url, folder="downloaded_pages"):
    # Create folder if it doesn't exist
    if not os.path.exists(folder):
        os.makedirs(folder)

    # Set up the Selenium WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in background without opening a browser window
    driver = webdriver.Chrome(options=options)

    try:
        # Fetch the page
        driver.get(url)

        # Save the page content
        filename = os.path.join(folder, url.replace("http://", "").replace("https://", "").replace("/", "_") + ".txt")
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(driver.page_source)

        # TODO: Add logic to find and process internal links for crawling subpages

    except Exception as e:
        print(f"Failed to process {url}: {e}")
    finally:
        driver.quit()

# Starting URL
start_url = "https://ee.co.uk/help"


fetch_and_save(start_url)


WebDriverException: Message: Service chromedriver unexpectedly exited. Status code was: 1


In [1]:
import requests
from bs4 import BeautifulSoup
import os

def fetch_and_save(url, folder="downloaded_pages"):
    # Create folder if it doesn't exist
    if not os.path.exists(folder):
        os.makedirs(folder)

    try:
        # Fetch content from URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes

        # Parse with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Save content to a file
        filename = os.path.join(folder, url.replace("http://", "").replace("https://", "").replace("/", "_") + ".txt")
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(soup.get_text())

        # Find and process all sub-page links
        for link in soup.find_all('a'):
            href = link.get('href')
            if href and href.startswith('http'):
                fetch_and_save(href, folder)

    except Exception as e:
        print(f"Failed to process {url}: {e}")

# Starting URL
start_url = "https://ee.co.uk/help"

fetch_and_save(start_url)


KeyboardInterrupt: 